# Applied Data Science Capstone
## Segmenting and Clustering Neighborhoods in Toronto

### Table of Contents
1. Webscraping using Beautiful soup to obtain Data from Wikipedia and transform into pandas Dataframe
2. Clean Dataset
3. Utilise Foursquare API to get latitude and longitude values of Toronto neighborhood
4. Use Geopy to latitiude and longitude of Toronoto
5. Use Folium to visualise Toronto Map with neighborhoods superimposed on top
6. Utilise Foursquare API to explore the neigbourhood and segment them

In [53]:
import numpy as np
import pandas as pd
import json
import xml
import requests

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

from bs4 import BeautifulSoup
!conda install -c conda-forge folium=0.5.0 --yes
import folium

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries Are Ready')

Solving environment: done

# All requested packages already installed.

Libraries Are Ready


## 1. Webscraping
Scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes.

Then transform the data into a pandas dataframe that consists of three columns: PostalCode, Borough, and Neighborhood

Ignore cells with a borough that is *Not assigned*.

In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html5lib')

In [31]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_pc = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_pc.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_pc.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 2. Clean Dataset
## 2a. Remove cells with a borough that is Not assigned

In [44]:
df_pc['Borough'].replace('Not assigned', np.nan, inplace=True)
df_pc.dropna(subset=['Borough'], inplace=True)
df_pc.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## 2b. Combine repeated neighbourhoods with comma

In [85]:
df_pcn = df_pc.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_pcn.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_pcn.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [35]:
df_pcn.shape

(103, 3)

## 3. Utilize the Foursquare location data to get the latitude and the longitude coordinates of each neighborhood

In [46]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']
df_geo.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

In [48]:
df_pos = pd.merge(df_pcn, df_geo, on=['Postcode'], how='inner')
df_tor = df_pos[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()
df_tor.head()

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


## 4. Use geopy library to get the latitude and longitude values of Toronto

In [61]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.653963, -79.387207.


## 5. Use Folium Library to visualise Toronto Map with neighborhoods superimposed on top

In [55]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 6.  Utilise Foursquare API to explore the neigbourhood and segment them

## 6a. Define Foursquare Credentials and Version

In [87]:
# The code was removed by Watson Studio for sharing.

## 6b. Explore the neigbourhood in Scarborough Borough
1. Slice and Cluster the neighborhoods in Scarborough
2. Slice and Create a new dataframe of the Scarborough

In [62]:
scarborough_data = df_tor[df_tor['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(10)

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029
7,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577
8,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476
9,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848


## 6c. Get Geographical Coordinates of Scarborough

In [67]:
address = 'Scarborough, Canada'

geolocator = Nominatim()
location_scarborough = geolocator.geocode(address)
latitude_scarborough = location_scarborough.latitude
longitude_scarborough = location_scarborough.longitude
print('The geographical coordinate of Scarborough are {}, {}.'.format(latitude_scarborough, longitude_scarborough))

The geographical coordinate of Scarborough are 43.773077, -79.257774.


## 6d. Visualise Scarborough with its neighborhood superimposed on it

In [68]:
map_scarb = folium.Map(location=[latitude_scarborough, longitude_scarborough], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

## 6e. Explore the first neighbourhood in the new datafame

In [71]:
scarborough_data.loc[0, 'Neighbourhood']

'Rouge, Malvern'

In [73]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


## 6e. Get top 100 venues that are in Rouge, Malvern within a radius of 500 metres
### First, let's create the GET request URL. Name your URL **url**.

In [79]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude_scarborough, 
    longitude_scarborough, 
    VERSION, 
    radius, 
    LIMIT)

### Send the GET request and examine the resutls

In [75]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cf246ac4434b9217ea48f3a'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5085ec39e4b0b1ead2eb0818-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/toys_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1f3941735',
         'name': 'Toy / Game Store',
         'pluralName': 'Toy / Game Stores',
         'primary': True,
         'shortName': 'Toys & Games'}],
       'id': '5085ec39e4b0b1ead2eb0818',
       'location': {'address': '300 Borough Drive',
        'cc': 'CA',
        'city': 'Scarborough',
        'country': 'Canada',
        'crossStreet': 'in Scarborough Town Centre',
        'distance': 284,
        'formattedAddress': ['300 Borough Drive (in Scarborough Town Centre)',
         'Scarborough ON M1P 4P5

### From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the *get_category_type* function from the Foursquare lab.

In [80]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [82]:
import json 
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649


In [84]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.
